# Modern Portfolio Theory

## Model

Modern Portfolio Theory has a number of assumptions which includes

* investors are risk averse, meaning that given two portfolios that offer the same expected return, investors will prefer the less risky one. 

* An investor will take on increased risk only if compensated by higher expected returns. 

* Conversely, an investor who wants higher expected returns must accept more risk. 

* The exact trade-off will be the same for all investors, but different investors will evaluate the trade-off differently based on individual risk aversion characteristics. 

* The implication is that a rational investor will not invest in a portfolio if a second portfolio exists with a more favorable risk-expected return profile – i.e., if for that level of risk an alternative portfolio exists that has better expected returns.

* There are 252 trading days in a calendar year on average.

### Modules

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import pandas_datareader as web
import scipy.stats as scs
import scipy.optimize as sco
import statsmodels.api as sm
import scipy.interpolate as sci
plt.style.use('ggplot')
np.random.seed(1479)

### Definitions

In [2]:
def normality_tests(arr):
    ''' Tests for normality distribution of given data set.
    Parameters
    ==========
    array: ndarray
    object to generate statistics on
    '''
    print('Skew of data set %14.3f' % scs.skew(arr))
    print('Skew test p-value %14.3f' % scs.skewtest(arr)[1])
    print('Kurt of data set %14.3f' % scs.kurtosis(arr))
    print('Kurt test p-value %14.3f' % scs.kurtosistest(arr)[1])
    print('Norm test p-value %14.3f' % scs.normaltest(arr)[1])

In [3]:
def print_statistics(array):
    ''' Prints selected statistics.
    Parameters
    ==========
    array: ndarray
    object to generate statistics on
    '''
    sta = scs.describe(array)
    print('%14s %15s' % ('statistic', 'value'))
    print(30 * '-')
    print('%14s %15.5f' % ('size', sta[0]))
    print('%14s %15.5f' % ('min', sta[1][0]))
    print('%14s %15.5f' % ('max', sta[1][1]))
    print('%14s %15.5f' % ('mean', sta[2]))
    print('%14s %15.5f' % ('std', np.sqrt(sta[3])))
    print('%14s %15.5f' % ('skew', sta[4]))
    print('%14s %15.5f' % ('kurtosis', sta[5]))

In [4]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf

In [5]:
def yprices(name, start, end):
    '''    
        Arguments: 
            name(string)    : input the ticker for the desired stock
            start(datetime) : Setting a starting date
            end(datetime)   : Setting a ending date 
         Returns:
            The function returns a Pandas DataFrame webscraped from yahoo finance containing the Date, High, Low, Close, Adj Close and Volume for the inquired stock 
            More information can be found at https://github.com/ranaroussi/fix-yahoo-finance
    '''
    yf.pdr_override() # Command to override the pandas search function
    return pdr.get_data_yahoo(name, start, end)

In [6]:
def statistics(weights):
 weights = np.array(weights)
 pret = np.sum(rets.mean() * weights) * 252
 pvol = np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))
 return np.array([pret, pvol, pret / pvol])

In [7]:
def min_func_sharpe(weights):
     return -statistics(weights)[2]

In [8]:
def min_func_variance(weights):
     return statistics(weights)[1] ** 2

In [9]:
def min_func_port(weights):
 return statistics(weights)[1]

### Data collecting

In [10]:
symbols = ['AAPL', 'MSFT', 'FB', 'DB', 'GLD']
noa = len(symbols)

In [11]:
start = datetime.datetime(2010, 1, 28)

In [12]:
end = datetime.datetime(2018, 10, 3)

Doing a webscrape of the yahoo price website for the quoted tickers, and lastly looking at the head and tail of the dataframe to ensure that we have the data needed for making the analysis and ensuring that we can optimize the porfolio

In [13]:
data = pd.DataFrame()
for sym in symbols:
 data[sym] = yprices(sym, start, end)['Adj Close']
data.columns = symbols
#loop that creates a dataframe
data.iloc[np.r_[0:4, -4:0]]

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


,AAPL,MSFT,FB,DB,GLD
Date,,,,,
2010-01-27,19.802147,23.597769,NaN,51.200840,106.529999
2010-01-28,18.983883,23.192135,NaN,50.284924,106.480003
2010-01-29,18.295170,22.412704,NaN,49.859688,105.959999
2010-02-01,18.549507,22.595629,NaN,51.887768,108.349998
2018-09-27,223.210526,113.433426,168.839996,11.810000,112.050003
2018-09-28,223.994431,113.393768,164.460007,11.360000,112.760002
2018-10-01,225.502670,114.623184,162.440002,11.250000,112.570000
2018-10-02,227.507050,114.167114,159.330002,11.130000,113.870003


### Data visualization and Analysis

The normal distribution can be considered the most important distribution in
finance and one of the major statistical building blocks of financial theory.
Among others, the following cornerstones of financial theory rest to a large
extent on the assumption that returns of a financial instrument are normally
distributed.

We see in the first figure

In [ ]:
(data).plot(figsize=(15, 5))
plt.legend(loc='upper left', fontsize=12)
plt.ylabel('prices in $')

Another way to plot this is plotting log daily returns (percent change compared to the day before). By plotting daily returns instead of actual prices, we can see the stocks’ volatility.

In [ ]:
rets = np.log(data / data.shift(1))
#new column for returns based on position above
#this uses log returns
rets.head(2)
#checking log returns

In [ ]:
plt.figure(figsize=(14, 7))
for c in rets.columns.values:
    plt.plot(rets.index, rets[c], lw=3, alpha=0.8,label=c)
plt.legend(loc='upper right', fontsize=12)
plt.ylabel('daily returns')

Amazon has two distinctive positive spikes and a couple of negative ones. Facebook has one highest positive spike. And Apple also has some spikes stand out from the plot. From the above plot, we can roughly see that Amazon looks like a quite risky stock, and Google seems to be the most stable one among them.

### Means, covariances and other measures

The annual return of the different stocks

In [ ]:
rets.mean() * 252
#simple arithmetic mean

The annual covariance matrix

In [ ]:
rets.cov()*252

#### Portfolio weights

In [ ]:
weights = np.random.random(noa)
#new numpy array of 4 random number that add up to 1
weights /= np.sum(weights)
#second line assures weight are equal to 1
weights

Expected return of i'th portfolio $\mathbb{E} R_{i}=w_{i, j}^{T} \times \mathbb{E} R_{j}$

In [ ]:
np.sum(rets.mean() * weights) * 252

Volatility of i'th portfolio $\sigma_{i}^{2}=w_{i, j}^{T} \times \operatorname{cov}\left(R_{j}\right) \times w_{i, j}$

In [ ]:
np.dot(weights.T, np.dot(rets.cov() * 252, weights))

Standard Deviations of the i'th portfolio $\sqrt{\sigma_{i}^{2}=w_{i, j}^{T} \times \operatorname{cov}\left(R_{j}\right) \times w_{i, j}}$

In [ ]:
np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights)))

### Monte Carlo simulation

In [ ]:
prets = []
pvols = []
#prets and vols are np.arrays
for p in range (20000):
 #loop underlines number of iterations for a random waiting
 weights = np.random.random(noa)
 weights /= np.sum(weights)
 #this makes it equal to 1
 prets.append(np.sum(rets.mean() * weights) * 252)
 #i get a value for each loop
 pvols.append(np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights))))
  #covariance value for each loop

prets = np.array(prets)
pvols = np.array(pvols)
#turns prets and pvols from lists into numpy arrays

#### Visualization

In [ ]:

plt.figure(figsize=(8, 5))
plt.scatter(pvols, prets, c = prets / pvols, marker='o', cmap='coolwarm')
# c is a calculated column of a sharpe ratio minus a risk free rate
# x-axis, y-axis, marker allows me to change the point style
plt.grid(True)
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio')

Now we build a function that, for a given vector of portfolio weights $w_i$, it
returns the portfolio’s expected return, volatility, and the Sharpe ratio
(as a vector).

Where the sharpe ratio is defined as the average return earned in excess of the risk-free rate per unit of volatility or total risk


$\textit{Sharpe Ratio}  =\frac{R_{p}-R_{f}}{\sigma_{p}}$

where

$\begin{aligned} R_{p} &=\textit { Return of portfolio } \\ R_{f} &=\textit { Risk-free rate } \\ \sigma_{p} &=\textit { Standard deviation of the portfolio's excess return } \end{aligned}$

In [ ]:
def statistics(weights):
 weights = np.array(weights)
 pret = np.sum(rets.mean() * weights) * 252 # annualized portfolio return
 pvol = np.sqrt(np.dot(weights.T, np.dot(rets.cov() * 252, weights))) # annualized portfolio variance
 return np.array([pret, pvol, pret / pvol]) # a tuble of annualized portfolio return, annualized portfolio variance and Sharpe Ratio of the portfolio

Importing the optimization module from Scipy

In [ ]:
import scipy.optimize as sco

Define a function for the negative Sharpe ratio
(we want to maximise it)

In [ ]:
def min_func_sharpe(weights):
 return -statistics(weights)[2]

Set up the constraint that portfolio weights add up to one $\sum^{n}_{i=1}w_i=1$

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
#this is a dictionary with two objects, that constrains the optimization

Set up boundaries for the portfolio weights (between 0 and 1)

In [ ]:
bnds = tuple((0, 1) for x in range(noa))

## Maximum Sharpe ratio portfolio

In [ ]:
%%time
opts = sco.minimize(min_func_sharpe, noa * [1. / noa,], method='SLSQP', bounds=bnds, constraints=cons)

In [ ]:
opts

Printing portfolio weights for the optimal sharpe portfolio allocation

In [ ]:
opts['x'].round(2)
#creates nparray based on key 'x' from the optimization, and rounds to 3 digit

Printing the expected return, variance and standard deviation from the assets in the portfolio based on their individual weights

In [ ]:
statistics(opts['x']).round(3)

Maximum Sharpe ratio portfolio properties

## Minimum variance portfolio

In [ ]:
%%time
optv = sco.minimize(min_func_variance, noa * [1. / noa,], method='SLSQP', bounds=bnds,
constraints=cons)

In [ ]:
optv

In [ ]:
optv['x'].round(3)

In [ ]:
statistics(optv['x']).round(3)

### EFFICIENT FRONTIER

Using Scipy to minimize the sharpe function

$\min_{w_{i}} w_{i}^{T} \Sigma w_{i}$

s.t

$w_{i}^{T} R=R_{\text { target }}$


where portfolio variances is defined as

 $w^TAw$= $\begin{bmatrix}w_1 \cdots w_n\end{bmatrix} \cdot \begin{bmatrix}
\sigma_{n1}^{2}& \cdots & \sigma_{1m}^{2}\\
\vdots     & \ddots & \vdots \\
\sigma_{n1}^{2}& \cdots & \sigma_{nm}^{2}\\
\end{bmatrix}
\cdot
\begin{bmatrix} 
w_1  \\
\vdots  \\
w_n  \\
\end{bmatrix}$ 



* We solve this problem for many levels of the target return.
* Each time we do, we get another point on the efficient frontier.
* We need to specify the constraint in a loop, as the target return
is always changing.

In [ ]:
cons = ({'type': 'eq', 'fun': lambda x: statistics(x)[0] - tret},
{'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0, 1) for x in weights)
#normal bounds for optimization

Define a range for target returns.

In [ ]:
%%time
trets = np.linspace(0.0, 0.25, 100)
#50 iterations, and the bounds
tvols = []
for tret in trets:
 cons = ({'type': 'eq', 'fun': lambda x: statistics(x)[0] - tret}, {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
 res = sco.minimize(min_func_port, noa * [1. / noa,], method='SLSQP', bounds=bnds, constraints=cons)
 tvols.append(res['fun'])
tvols = np.array(tvols)

#### Visualization

In [ ]:
plt.figure(figsize=(12, 4))
plt.scatter(pvols, prets,
 c=prets / pvols, marker='|', cmap ='coolwarm')
 # random portfolio composition
plt.scatter(tvols, trets,
 c=trets / tvols, marker='x', cmap='coolwarm')
 # efficient frontier
plt.plot(statistics(opts['x'])[1], statistics(opts['x'])[0],
 'r*', markersize=15.0)
 # portfolio with highest Sharpe ratio
plt.plot(statistics(optv['x'])[1], statistics(optv['x'])[0],
 'y*', markersize=15.0)
 # minimum variance portfolio
plt.grid(True)
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio')

# CAPITAL MARKET LINE

In [ ]:
import scipy.interpolate as sci

The trick is to remember that the return vector is sorted in ascending
order.
Portfolios with returns above the minimum variance portfolio are
non-dominated!

In [ ]:
ind = np.argmin(tvols)
evols = tvols[ind:]
erets = trets[ind:]

In [ ]:
tck = sci.splrep(evols, erets)

In [ ]:
def f(x):
#Efficient frontier function (splines approximation)
 return sci.splev(x, tck, der=0)
def df(x):
#First derivative of efficient frontier function
 return sci.splev(x, tck, der=1)

$\begin{aligned} t(x) &=a+b \cdot x \\ t(0) &=r_{f} \\ t(x) &=f(x) \\ t(x) &=f(x) & \Leftrightarrow \quad b \end{aligned}=f(x)$

In [ ]:
def equations(p, rf=0):
 eq1 = rf - p[0]
 eq2 = rf + p[1] * p[2] - f(p[2])
 eq3 = p[1] - df(p[2])
 return eq1, eq2, eq3

In [ ]:
opt = sco.fsolve(equations, [0.01, 0.5, 0.15])

In [ ]:
opt

In [ ]:
np.round(equations(opt), 3)

In [ ]:
plt.figure(figsize=(8, 4))
plt.scatter(pvols, prets,
 c=(prets - 0.01) / pvols, marker='o', cmap='coolwarm')
 # random portfolio composition
plt.plot(evols, erets, 'black', lw=4.0)
 # efficient frontier
cx = np.linspace(0.0, 0.3)
plt.plot(cx, opt[0] + opt[1] * cx, lw=1.5)
 # capital market line
plt.plot(opt[2], f(opt[2]), 'r*', markersize=15.0)
plt.grid(True)
plt.axhline(0, color='k', ls='-', lw=2.0)
plt.axvline(0, color='k', ls='-', lw=2.0)
plt.xlabel('expected volatility')
plt.ylabel('expected return')
plt.colorbar(label='Sharpe ratio')

### The optimal portfolio allocation